In [3]:
!conda info



     active environment : GNNGeneDisease
    active env location : /home/indradutta/miniconda3/envs/GNNGeneDisease
            shell level : 2
       user config file : /home/indradutta/.condarc
 populated config files : 
          conda version : 23.9.0
    conda-build version : not installed
         python version : 3.11.4.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=12.0=0
                          __glibc=2.35=0
                          __linux=5.19.0=0
                          __unix=0=0
       base environment : /home/indradutta/miniconda3  (writable)
      conda av data dir : /home/indradutta/miniconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cac

In [9]:
cd GNN_GeneDisease/Disease_gene_prioritization_GCN/


/home/indradutta/MSc_Lab_1/GNN_GeneDisease/Disease_gene_prioritization_GCN


In [15]:
!python3 main_prioritization.py

2023-10-14 21:41:19.059679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 21:41:19.059726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 21:41:19.059763: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
>>>>>> Before Main
Edge types: 6
>>>>> Inside Main
Defining placeholders
Create minibatch iterator
/home/indradutta/MSc_Lab_1/GNN_GeneDisease/Disease_gene_prioritization_GCN/decagon/deep/minibatch.py:61: RuntimeWarning: divide by zero encountered in power
  rowdegree_mat_inv = sp.diags(np.nan_to_num(np.power(rowsum, -0.5)).flatten())
/home/indradutta/MSc_Lab_1